In [ ]:
from sage.all import *
from sage.combinat.posets.moebius_algebra import BasisAbstract, MoebiusAlgebraBases, QuantumMoebiusAlgebra

def reduced_characteristic_polynomial(M, var='x'):
    """
    Calculates the reduced characteristic polynomial of a matroid M.

    This is defined as chi(M, x) / (x-1) for a non-empty, loopless matroid,
    and -1 for the empty matroid.

    Args:
        M: A SageMath matroid object.
        var: The name for the polynomial variable.

    Returns:
        A polynomial over the Rational Field.
    """
    R = PolynomialRing(QQ, var)
    t = R.gen()

    # Per Definition 2.4, the reduced characteristic polynomial of the empty
    # matroid is -1.
    if M.rank() == 0 and not M.groundset(): # U_{0,0}
        return R(-1)

    # If the matroid has loops, its characteristic polynomial is 0.
    if M.has_loops():
        return R(0)

    # For a non-empty, loopless matroid, it is chi(x)/(x-1).
    chi = M.characteristic_polynomial(var=var)
    return chi // (t - 1)


@cached_function
def chow_polynomial(M, var='x'):
    """
    Calculates the Chow polynomial (the Hilbert-Poincaré series of the
    Chow ring) of a matroid M using the recursion from Theorem 3.12.

    Args:
        M: A SageMath matroid object.
        var: The name for the polynomial variable.

    Returns:
        The Chow polynomial of M.
    """
    R = PolynomialRing(QQ, var)

    # Base case: If rk(M) = 0, the polynomial is 1.
    if M.rank() == 0:
        return R(1)

    total_sum = R(0)
    flats = M.lattice_of_flats()
    
    # The recursion sums over all non-empty flats F.
    for F_set in flats:
        if not F_set: # Skip the empty flat
            continue

        # For each flat, we need the restriction M|_F and contraction M/F.
        M_restriction = M.restriction(F_set)
        M_contraction = M.contraction(F_set)

        # Calculate the two components of the summand.
        h_bar = reduced_characteristic_polynomial(M_restriction, var)
        H_M_slash_F = chow_polynomial(M_contraction, var) # Recursive call

        total_sum += h_bar * H_M_slash_F

    return total_sum

class ZetaBasis(BasisAbstract):
    """
    The zeta basis for the Quantum Möbius Algebra.
    """
    def __init__(self, M, prefix='Z') -> None:
        self._basis_name = "zeta"
        # This now works because the parent algebra M is a QuantumMoebiusAlgebra
        CombinatorialFreeModule.__init__(self, M.base_ring(),
                                        tuple(M._lattice),
                                        prefix=prefix,
                                        category=MoebiusAlgebraBases(M))

        E = M.E()
        phi = self.module_morphism(self._to_natural_basis,
                                   codomain=E,
                                   triangular='upper', unitriangular=True,
                                   key=M._lattice._element_to_vertex)

        phi.register_as_coercion()
        (~phi).register_as_coercion()

    def _to_natural_basis(self, x):
        M = self.realization_of()
        L = M._lattice
        E = M.E()
        q = M._q
        rank = L.rank_function()
        return E.sum_of_terms((y, q**(rank(x) - rank(y)) *
                               L.kazhdan_lusztig_polynomial(y, x)(q=q**-2))
                              for y in L.order_ideal([x]))

class ChowBasis(BasisAbstract):
    """
    The Chow basis for the Quantum Möbius Algebra.
    """
    def __init__(self, M, prefix='C') -> None:
        self._basis_name = "chow"
        # This now works because the parent algebra M is a QuantumMoebiusAlgebra
        CombinatorialFreeModule.__init__(self, M.base_ring(),
                                        tuple(M._lattice),
                                        prefix=prefix,
                                        category=MoebiusAlgebraBases(M))

        E = M.E()
        phi = self.module_morphism(self._to_natural_basis,
                                   codomain=E,
                                   triangular='upper', unitriangular=True,
                                   key=M._lattice._element_to_vertex)

        phi.register_as_coercion()
        (~phi).register_as_coercion()

    def _to_natural_basis(self, x):
        M = self.realization_of()
        L = M._lattice
        E = M.E()
        q = M._q
        rank = L.rank_function()
        return E.sum_of_terms((y, q**(rank(x) - rank(y)) *
                               L.chow_polynomial(y, x).subs({q: q**-2}))
                              for y in L.order_ideal([x]))

# Inherit from QuantumMoebiusAlgebra
class DeformedMoebiusAlgebra(QuantumMoebiusAlgebra):
    def __init__(self, L):
        self._R = LaurentPolynomialRing(ZZ, 'x')
        x = self._R.gen()
        # Call the super constructor with the correct arguments for QuantumMoebiusAlgebra
        super().__init__(L, x)
        self._zeta_basis = ZetaBasis(self)
        self._chow_basis = ChowBasis(self)

    def zeta(self):
        """
        Return the zeta basis for this algebra.
        """
        return self._zeta_basis

    def chow(self):
        """
        Return the Chow basis for this algebra.
        """
        return self._chow_basis

    def is_in_Hp(self, alpha, E, t):
        rank_func = lambda x : self._L.rank(x)
        alpha_dict = E(alpha).monomial_coefficients()
        for F in alpha_dict.keys():
            sum_S_F = 0
            rank_F = rank_func(F)
            for G in self._L.order_filter([F]):
                coeff_G = alpha_dict.get(G, 0)
                if coeff_G != 0:
                    rank_G = rank_func(G)
                    term = (t**(rank_F - rank_G)) * coeff_G
                    sum_S_F += term
            try:
                if sum_S_F != sum_S_F.subs({t: 1/t}):
                    print(f"Palindromic check failed for F={F}")
                    return False
            except Exception as e:
                print(f"Error during palindromic check for F={F}: {e}")
                return False
        return True

In [18]:

M = matroids.Uniform(2, 4)
L = M.lattice_of_flats()
D = DeformedMoebiusAlgebra(L)
chow_basis = D.chow().basis()
keys = chow_basis.keys()
nat_basis = D.E()
nat_basis(chow_basis[keys[0]]).monomial_coefficients()

AttributeError: 'FiniteLatticePoset_with_category' object has no attribute 'chow_polynomial'